In [1]:
import numpy as np
import pickle
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
folder_path = './data/motion/smplx/train'

In [3]:
train_file_names = (sorted([f for f in os.listdir(folder_path) if f.endswith('.npy')]))

In [4]:
train_orient = []
train_betas = []

In [5]:
for i in tqdm(range(len(train_file_names)//2)):
    follower_file = np.load(os.path.join(folder_path, train_file_names[i*2]), allow_pickle=True).item()
    leader_file = np.load(os.path.join(folder_path, train_file_names[i*2+1]), allow_pickle=True).item()

    assert follower_file['transl'].shape[0] == leader_file['transl'].shape[0], "check the shape of the files"

    length = leader_file['transl'].shape[0]

    follower_file_processed = np.zeros((length, 3+165+3))
    leader_file_processed = np.zeros((length, 3+165+3))
    follower_beta = np.zeros((length, 16))
    leader_beta = np.zeros((length, 16))

    for data in ['transl', 'poses', 'global_orient', 'betas']:
        if data == 'transl':
            if (follower_file[data].shape[1] != 3 or leader_file[data].shape[1] != 3): print('error transl shape')
            follower_file_processed[:, :3] = follower_file[data]
            leader_file_processed[:, :3] = leader_file[data]
        elif data == 'global_orient':
            if (follower_file[data].shape[1] != 3 or leader_file[data].shape[1] != 3): print('error global_orient shape')
            follower_file_processed[:, -3:] = follower_file[data]
            leader_file_processed[:, -3:] = leader_file[data]
        elif data == 'betas':
            if (follower_file[data].shape[1] != 16 or leader_file[data].shape[1] != 16): print('error betas shape')
            follower_beta = follower_file[data]
            leader_beta = leader_file[data]
        else:
            if (follower_file[data].shape[1] != 165 or leader_file[data].shape[1] != 165): print('error poses shape')
            follower_file_processed[:, 3:168] = follower_file[data]
            leader_file_processed[:, 3:168] = leader_file[data]
        
       
    
    for j in range(0, length, 300):
        f_j = follower_file_processed[j:j+300]
        l_j = leader_file_processed[j:j+300]

        f_b = follower_beta[j:j+300]
        l_b = leader_beta[j:j+300]

        d_b = (f_b, l_b)
        d_j = (f_j, l_j)

        train_orient.append(d_j)
        train_betas.append(d_b)

100%|██████████| 133/133 [00:01<00:00, 113.75it/s]


In [6]:
for i in range(len(train_orient)):
    length = train_orient[i][0].shape[0]
    if length!=300:
        follower_orient = np.zeros((300, 171))
        leader_orient = np.zeros((300, 171))
        follower_beta = np.zeros((300, 16))
        leader_beta = np.zeros((300, 16))

        follower_orient[:length] = train_orient[i][0]
        leader_orient[:length] = train_orient[i][1]

        follower_beta[:length] = train_betas[i][0]
        leader_beta[:length] = train_betas[i][1]

        train_orient[i] = (follower_orient, leader_orient)
        train_betas[i] = (follower_beta, leader_beta)

In [10]:
folder_path = './data/motion/smplx/test'

In [11]:
test_file_names = (sorted([f for f in os.listdir(folder_path) if f.endswith('.npy')]))

In [12]:
test_orient = []
test_betas = []

In [13]:
for i in tqdm(range(len(test_file_names)//2)):
    follower_file = np.load(os.path.join(folder_path, test_file_names[i*2]), allow_pickle=True).item()
    leader_file = np.load(os.path.join(folder_path, test_file_names[i*2+1]), allow_pickle=True).item()

    assert follower_file['transl'].shape[0] == leader_file['transl'].shape[0], "check the shape of the files"

    length = leader_file['transl'].shape[0]

    follower_file_processed = np.zeros((length, 3+165+3))
    leader_file_processed = np.zeros((length, 3+165+3))
    follower_beta = np.zeros((length, 16))
    leader_beta = np.zeros((length, 16))

    for data in ['transl', 'poses', 'global_orient', 'betas']:
        if data == 'transl':
            if (follower_file[data].shape[1] != 3 or leader_file[data].shape[1] != 3): print('error transl shape')
            follower_file_processed[:, :3] = follower_file[data]
            leader_file_processed[:, :3] = leader_file[data]
        elif data == 'global_orient':
            if (follower_file[data].shape[1] != 3 or leader_file[data].shape[1] != 3): print('error global_orient shape')
            follower_file_processed[:, -3:] = follower_file[data]
            leader_file_processed[:, -3:] = leader_file[data]
        elif data == 'betas':
            if (follower_file[data].shape[1] != 16 or leader_file[data].shape[1] != 16): print('error betas shape')
            follower_beta = follower_file[data]
            leader_beta = leader_file[data]
        else:
            if (follower_file[data].shape[1] != 165 or leader_file[data].shape[1] != 165): print('error poses shape')
            follower_file_processed[:, 3:168] = follower_file[data]
            leader_file_processed[:, 3:168] = leader_file[data]
        
       
    
    for j in range(0, length, 300):
        f_j = follower_file_processed[j:j+300]
        l_j = leader_file_processed[j:j+300]

        f_b = follower_beta[j:j+300]
        l_b = leader_beta[j:j+300]

        d_b = (f_b, l_b)
        d_j = (f_j, l_j)

        test_orient.append(d_j)
        test_betas.append(d_b)

100%|██████████| 34/34 [00:00<00:00, 103.43it/s]


In [14]:
for i in range(len(test_orient)):
    length = test_orient[i][0].shape[0]
    if length!=300:
        follower_orient = np.zeros((300, 171))
        leader_orient = np.zeros((300, 171))
        follower_beta = np.zeros((300, 16))
        leader_beta = np.zeros((300, 16))

        follower_orient[:length] = test_orient[i][0]
        leader_orient[:length] = test_orient[i][1]

        follower_beta[:length] = test_betas[i][0]
        leader_beta[:length] = test_betas[i][1]

        test_orient[i] = (follower_orient, leader_orient)
        test_betas[i] = (follower_beta, leader_beta)

In [19]:
# save the data
with open('train_orient.pkl', 'wb') as f:
    pickle.dump(train_orient, f)

with open('train_betas.pkl', 'wb') as f:
    pickle.dump(train_betas, f)

with open('test_orient.pkl', 'wb') as f:
    pickle.dump(test_orient, f)

with open('test_betas.pkl', 'wb') as f:
    pickle.dump(test_betas, f)

print('Data saved successfully')

Data saved successfully
